### Steady State

#### Inclined Slider Use case

- non-periodic
- density boundary conditions left and right
- no energy included

To activate FEM:

```yaml
numerics:
    solver: fem
```

To select the terms to include (see `Equations` section in [FEM theory](./08_FEM1d_theory.ipynb)):

```yaml
fem_solver:
    equations:
        term_list: ['R11', 'R11S', 'R21', 'R24']
```

When solving steady state, an internal convergence loop is used.

In [ ]:
# non-periodic boundary conditions and steady-state; inclined slider use case
%matplotlib inline
from GaPFlow import Problem

sim = """
options:
    output: data/inclined
    write_freq: 1000
    silent: False
grid:
    Lx: 0.1
    Ly: 1.
    Nx: 100
    Ny: 1
    xE: ['D', 'N', 'N']
    xW: ['D', 'N', 'N']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
    xE_D: 1.1853
    xW_D: 1.1853
geometry:
    type: inclined
    hmax: 6.6e-5
    hmin: 1.0e-5
    U: 50.
    V: 0.
numerics:
    solver: fem
    tol: 1e-6
    dt: 1e-8
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    equations:
        term_list: ['R11', 'R11S', 'R21', 'R24']
"""

problem = Problem.from_string(sim)
problem.run()
problem.plot()

### Dynamic

#### Journal Use Case

Activate dynamic mode (default is False):

```yaml
fem_solver:
    dynamic: True
```

Note that in the dynamic case

```yaml
numerics:
    tol: 1e-8
    max_it: 5
```

applies to the problem main loop and

```yaml
fem_solver:
    R_norm_tol: 1e-04
    max_iter: 10
```

applies to the inner loop that needs to converge for each time step.

In [ ]:
# periodic boundary conditions and time-stepping; journal use case
%matplotlib inline
from GaPFlow import Problem

sim = """
options:
    output: data/journal
    write_freq: 1
    silent: False
grid:
    dx: 1.e-5
    dy: 1.
    Nx: 100
    Ny: 1
    xE: ['P', 'P', 'P']
    xW: ['P', 'P', 'P']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
geometry:
    type: journal
    CR: 1.e-2
    eps: 0.7
    U: 0.1
    V: 0.
numerics:
    solver: fem
    tol: 1e-8
    dt: 1e-02
    max_it: 5
properties:
    shear: 0.0794
    bulk: 0.
    EOS: DH
    P0: 101325.
    rho0: 877.7007
    T0: 323.15
    C1: 3.5e10
    C2: 1.23
fem_solver:
    type: newton_alpha
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-04
    equations:
        term_list: ['R11', 'R11S', 'R21', 'R24', 'R1T', 'R2T']
"""

problem = Problem.from_string(sim)
problem.run()
problem.plot()

### Energy

#### Energy - Shear Heat

Dynamic simulation of Couette flow including the *wall stress work* term from the `energy` equations.

Observe how the mean total energy (and temperature) rises due to work done by the movement of the wall.

Inclusion of the wall stress and time derivative terms of the energy equation. Note that all other energy terms such as heat exchange with the wall are not included here.

```yaml
['R34', 'R3T']
```

Relevant for this case is also the volume-specific heat capacity $c_v$, which determines the temperature increase.

```yaml
energy_spec:
    cv: 1005.
```


In [ ]:
# shear wall heating
%matplotlib inline
from GaPFlow import Problem

sim = """
options:
    output: data/journal
    write_freq: 1
    silent: False
grid:
    dx: 1.e-5
    dy: 1.
    Nx: 100
    Ny: 1
    xE: ['P', 'P', 'P']
    xW: ['P', 'P', 'P']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
geometry:
    type: inclined
    hmax: 1.0e-5
    hmin: 1.0e-5
    U: 1.
    V: 0.
numerics:
    solver: fem
    tol: 0
    dt: 1e-5
    max_it: 20
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-04
    equations:
        energy: True
        term_list: ['R11', 'R11S', 'R1T', 'R21', 'R24', 'R2T', 'R34', 'R3T']
energy_spec:
    cv: 1005.
"""
import numpy as np

E_mean_history = []
T_mean_history = []

def compute_means(args):
    E_mean = np.mean(problem.energy.energy)
    E_mean_history.append(E_mean)
    T_mean = np.mean(problem.energy.temperature)
    T_mean_history.append(T_mean)

problem = Problem.from_string(sim)
problem.solver.num_solver.callback_fun = compute_means
problem.run()
problem.animate()

#### Energy - Wall Heat

Dynamic simulation of heat diffusion from a hot fluid into cold walls using the Robin boundary condition (`Tz_Robin` wall model).

Setup:
- Periodic boundary conditions (uniform in x)
- No wall movement (U=0, V=0)
- Constant channel height
- Initial fluid temperature: 400 K
- Wall temperature: 300 K

The Robin boundary condition models heat flux at the wall interface:

```yaml
energy_spec:
    wall_flux_model: Tz_Robin
    T_wall: 300.
    h_Robin: 1e4
    k: 0.026
    cv: 1005.
```

This use case tracks the evolution of the temperature profile T(z) across the channel height using a callback function. The `get_T_z` function from `GaPFlow.models.heatflux` computes the parabolic temperature profile based on the current state variables.

In [ ]:
# wall heat diffusion with T(z) tracking
%matplotlib inline
import numpy as np
from GaPFlow import Problem
from GaPFlow.models.heatflux import get_T_z_at_cell

sim = """
options:
    output: data/wall_heat
    write_freq: 5
    silent: False
grid:
    dx: 1.e-5
    dy: 1.
    Nx: 10
    Ny: 1
    xE: ['P', 'P', 'P']
    xW: ['P', 'P', 'P']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
geometry:
    type: inclined
    hmax: 1.0e-3
    hmin: 1.0e-3
    U: 0.
    V: 0.
numerics:
    solver: fem
    tol: 0
    dt: 1e-4
    max_it: 100
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-4
    equations:
        energy: True
        term_list: ['R11', 'R11S', 'R1T', 'R21', 'R24', 'R2T', 'R36', 'R3T']
energy_spec:
    cv: 1005.
    k: 0.026
    wall_flux_model: Tz_Robin
    T_wall: 300.
    T0: 400.
    h_Robin: 1e-4
"""

# Initialize problem
problem = Problem.from_string(sim)

# Track T(z) profile via callback
i, j = 1, 1
z = np.linspace(0, problem.topo.h[i, j], 21)
T_z_history = {'iteration': [], 'T_z': []}

def track_T_z(args):
    """Callback to track T(z) profile evolution."""
    T_z = get_T_z_at_cell(problem, i, j, z)
    T_z_history['iteration'].append(len(T_z_history['T_z']))
    T_z_history['T_z'].append(T_z.copy())

problem.solver.num_solver.callback_fun = track_T_z

# Run simulation
problem.run()
problem.animate()

In [ ]:
# Plot T(z) evolution
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 4))

iterations = T_z_history['iteration']
T_z_data = T_z_history['T_z']
dt = problem.numerics['dt']

# Plot every nth profile to avoid clutter
n_profiles = min(10, len(T_z_data))
indices = np.linspace(0, len(T_z_data) - 1, n_profiles, dtype=int)

colors = plt.cm.coolwarm(np.linspace(1, 0, n_profiles))

for i, idx in enumerate(indices):
    time = iterations[idx] * dt
    ax.plot(T_z_data[idx], z * 1e6, color=colors[i], label=f't = {time:.2e} s')

ax.axvline(x=300, color='k', linestyle='--', alpha=0.5, label='T_wall = 300 K')
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('z [μm]')
ax.set_title('Temperature Profile T(z) Evolution')
ax.legend(loc='best', fontsize=8)
ax.grid(True, alpha=0.3)
plt.show()

#### Energy - Shear and Wall

Dynamic simulation combining shear heating from wall motion with heat transfer to the walls using the Robin boundary condition.

Setup:
- Periodic boundary conditions (Couette flow)
- Wall movement: U > 0 (shear heating source)
- Constant channel height
- Initial fluid temperature: 290 K (cold fluid)
- Wall temperature: 300 K (warm walls)

This demonstrates the balance between:
1. **Shear heating (R34)**: Energy input from viscous dissipation due to wall motion
2. **Wall heat flux (R36)**: Energy exchange with the walls via Robin BC

The system should reach a thermal equilibrium where shear heating balances the heat flux to the walls. With the initial fluid colder than the walls, we expect:
- Initial heating from both shear work and wall heat transfer
- As fluid heats up past wall temperature, wall heat flux reverses direction
- Final equilibrium temperature above wall temperature (due to shear heating)

```yaml
fem_solver:
    equations:
        energy: True
        term_list: ['R11', 'R11S', 'R1T', 'R21', 'R24', 'R2T', 'R34', 'R36', 'R3T']
```

In [ ]:
# Shear heating with wall heat transfer - equilibrium demonstration
%matplotlib inline
import numpy as np
from GaPFlow import Problem

sim = """
options:
    output: data/shear_wall
    write_freq: 5
    silent: False
grid:
    dx: 1.e-5
    dy: 1.
    Nx: 10
    Ny: 1
    xE: ['P', 'P', 'P']
    xW: ['P', 'P', 'P']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
geometry:
    type: inclined
    hmax: 1.0e-3
    hmin: 1.0e-3
    U: 50.
    V: 0.
numerics:
    solver: fem
    tol: 0
    dt: 1e-3
    max_it: 100
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-4
    equations:
        energy: True
        term_list: ['R11', 'R11S', 'R1T', 'R21', 'R24', 'R2T', 'R34', 'R36', 'R3T']
energy_spec:
    cv: 1005.
    k: 0.026
    wall_flux_model: Tz_Robin
    T_wall: 300.
    T0: 290.
    h_Robin: 1e-4
"""

# Initialize problem
problem = Problem.from_string(sim)

# Track temperature history via callback
T_history = {'iteration': [], 'T_mean': []}

def track_temperature(args):
    """Callback to track temperature evolution."""
    T = problem.energy.temperature
    T_history['iteration'].append(len(T_history['T_mean']))
    T_history['T_mean'].append(np.mean(T[1:-1,1:-1]))

problem.solver.num_solver.callback_fun = track_temperature

# Run simulation
problem.run()
problem.animate()

In [ ]:
# Plot temperature evolution to equilibrium
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 3))

iterations = np.array(T_history['iteration'])
T_mean = np.array(T_history['T_mean'])
T_init = problem.energy_spec['T0'][1]
dt = problem.numerics['dt']
time = iterations * dt

ax.plot(time, T_mean, 'b-', linewidth=1.5, label='T_fluid average')
ax.axhline(y=300, color='r', linestyle='--', linewidth=1., label='T_wall = 300 K')
ax.axhline(y=T_init, color='g', linestyle='--', linewidth=1., label=f'T_init = {T_init} K')

ax.set_xlabel('Time [s]')
ax.set_ylabel('Temperature [K]')
ax.set_title('Balance of Shear Heating vs Wall Heat Transfer')
ax.legend(loc='best')
ax.grid(False)

plt.show()

#### Energy - Cooling

Dynamic simulation of a 'cooling rod' with non-periodic boundary conditions.

Initial and boundary conditions for temperature can be specified as follows:

```yaml
energy_spec:
    T0: [half_sine, 300, 400]
    bc_xW: D
    bc_xE: D
    T_bc_xW: 300.
    T_bc_xE: 300.
```

In [ ]:
# Cooling rod with non-periodic BC and half-sine initial temperature
%matplotlib inline
import numpy as np
from GaPFlow import Problem

sim = """
options:
    output: data/cooling
    write_freq: 5
    silent: False
grid:
    Lx: 0.1
    Ly: 1.
    Nx: 50
    Ny: 1
    xE: ['D', 'N', 'N']
    xW: ['D', 'N', 'N']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
    xE_D: 1.1853
    xW_D: 1.1853
geometry:
    type: inclined
    hmax: 1.0e-3
    hmin: 1.0e-3
    U: 0.
    V: 0.
numerics:
    solver: fem
    tol: 0
    dt: 5e-01
    max_it: 100
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-7
    equations:
        energy: True
        term_list: ['R11', 'R11S', 'R1T', 'R21', 'R24', 'R2T', 'R35', 'R3T']
energy_spec:
    cv: 1005.
    k: 0.026
    T0: [half_sine, 300, 400]
    bc_xW: D
    bc_xE: D
    T_bc_xW: 300.
    T_bc_xE: 300.
"""

problem = Problem.from_string(sim)
problem.run()
problem.animate()